## 导包

In [25]:
import docx
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.stats import wasserstein_distance

from gensim.models import Word2Vec
import numpy as np

import re
import os

## 新华网爬虫

In [15]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta


In [ ]:
# 获取新闻详情
def get_news_detail(news_url):
    response = requests.get(news_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    title = soup.find('h1').text.strip()
    content = ' '.join([p.text.strip() for p in soup.find_all('p') if p.text.strip()])

    return title, content

# 保存为Word文档
def save_to_word(file_path, title, content):
    doc = docx.Document()
    doc.add_heading(title, level=1)
    doc.add_paragraph(content)
    doc.save(file_path)

# 爬取新闻
def scrape_news(base_url, categories, save_path, months=3):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=30*months)

    if not os.path.exists(save_path):
        os.makedirs(save_path)

    for category in categories:
        category_path = os.path.join(save_path, category)
        if not os.path.exists(category_path):
            os.makedirs(category_path)
        
        category_url = f"{base_url}/{category}"
        response = requests.get(category_url)
        soup = BeautifulSoup(response.content, 'html.parser')

        news_links = soup.find_all('a', href=True)
        
        for link in news_links:
            news_url = link['href']
            # 假设新闻详情页面包含发布日期信息
            try:
                title, content = get_news_detail(news_url)
                # 假设我们可以从详情页面中提取发布日期
                pub_date = datetime.strptime(soup.find('time').text.strip(), '%Y-%m-%d')
                if start_date <= pub_date <= end_date:
                    file_path = os.path.join(category_path, f"{title}.docx")
                    save_to_word(file_path, title, content)
            except Exception as e:
                print(f"Failed to process {news_url}: {e}")


In [ ]:

# 示例用法
# base_url = 'http://www.xinhuanet.com'
# categories = ['politics', 'world', 'sports']  # 替换为实际的新闻分类
# save_path = 'news_data'  # 保存路径
# scrape_news(base_url, categories, save_path)

## utils

### 读取文档内容

In [13]:
# 读取Word文档内容
def read_word_doc(file_path):
    print('\n-----------------读取Word文档内容-----------------')
    doc = docx.Document(file_path)
    full_text = []
    for paragraph in doc.paragraphs:
        full_text.append(paragraph.text)

    text = ' '.join(full_text)
    print('去除空格和标点符号前的文本长度：', len(text))

    text = re.sub(r'\s+', '', text)  # 去除所有空格
    text = re.sub(r'[^\w]', '', text)  # 去除所有标点符号
    print('去除空格和标点符号后的文本长度：', len(text))
    print('文本内容:', text)

    return text


### 向量表示

In [31]:
# 计算文本的TF-IDF向量表示
def compute_tfidf_vector(texts):
    print('\n-----------------计算文本的TF-IDF向量表示-----------------')

    # 使用 jieba 进行分词
    def chinese_tokenizer(text):
        return jieba.lcut(text)

    # 初始化 TfidfVectorizer，使用自定义的分词器
    vectorizer = TfidfVectorizer(
        tokenizer=chinese_tokenizer,  # 自定义分词器
        stop_words=None,              # 可以提供中文停用词列表
        lowercase=False,              # 保持原始大小写
        ngram_range=(1, 2)            # 只使用一元组
    )
    
    tfidf_matrix = vectorizer.fit_transform(texts)
    tfidf_matrix_array = tfidf_matrix.toarray()
    print('TF-IDF矩阵的形状：', tfidf_matrix_array.shape)
    # 打印特征名称（单词和短语）
    print('打印特征名称：', vectorizer.get_feature_names_out())

    return tfidf_matrix_array, vectorizer

# 计算文本的Word2Vec向量表示
def compute_word2vec_vector(texts):
    # print('\n-----------------计算文本的Word2Vec向量表示-----------------')
    tokenized_texts = [text.split() for text in texts]
    # print('Tokenized Texts Length:', len(tokenized_texts))

    model = Word2Vec(tokenized_texts, vector_size=100, window=5, min_count=1, workers=4)
    vectors = [np.mean([model.wv[word] for word in text if word in model.wv] or [np.zeros(100)], axis=0) for text in tokenized_texts]
    # print('Word2Vec Vectors Shape:', np.array(vectors).shape)
    return vectors


In [32]:
# 计算两个文本的Wasserstein距离
def calculate_wasserstein_distance(vector1, vector2):
    return wasserstein_distance(vector1, vector2)

In [37]:
u = [0.2329]
v = [0.2485]

distance = wasserstein_distance(u, v)

print(f'Wasserstein 距离: {distance}')

Wasserstein 距离: 0.015600000000000003


## Main

### 两两计算相似度

In [35]:
# 计算两个文本的相似度
def similarityCalculation_two_texts(file_path1, file_path2):
    # 读取Word文档内容
    text1 = read_word_doc(file_path1)
    text2 = read_word_doc(file_path2)

    # 计算TF-IDF向量表示
    tfidf_vectors, _ = compute_tfidf_vector([text1, text2])

    # 计算Word2Vec向量表示
    word2vec_vectors = compute_word2vec_vector([text1, text2])
    # print('Word2Vec Vectors:', word2vec_vectors)  # (2, 100)

    # 用TF-IDF向量计算Wasserstein距离
    distance_tfidf = calculate_wasserstein_distance(tfidf_vectors[0], tfidf_vectors[1])

    # 用Word2Vec向量计算Wasserstein距离
    distance_w2v = calculate_wasserstein_distance(word2vec_vectors[0], word2vec_vectors[1])

    print(f'\nThe Wasserstein distance based on tfidf between the two texts is: {distance_tfidf}')
    print(f'\nThe Wasserstein distance based on word2vector between the two texts is: {distance_w2v}')

In [36]:
# main
file_path1 = 'news\财经\科技金融探索服务新路径.docx'
file_path2 = 'news_AI\财经\科技金融探索服务新路径.docx'

similarityCalculation_two_texts(file_path1, file_path2)


-----------------读取Word文档内容-----------------
去除空格和标点符号前的文本长度： 3150
去除空格和标点符号后的文本长度： 2828
文本内容: 作为推动科技创新的重要引擎科技金融是中央金融工作会议提出的五篇大文章之一近日国家金融监督管理总局党委书记局长李云泽表示做好五篇大文章是金融业责无旁贷的重要使命银行保险机构要立足职责定位深入探索金融服务新质生产力的新路径新模式开发适配性强的金融产品提高金融服务水平全力支持科技创新绿色发展新兴产业和未来产业目前金融服务科创企业还存在哪些难点如何引导金融呵护科创企业发展加大信贷供给今年以来国家金融监督管理总局会同相关部门不断完善金融支持科技创新的政策丰富金融支持工具引导银行业加力服务科技型企业无论是信贷产品创新还是一揽子金融服务模式都旨在提升金融服务科创企业的针对性有效性为扎实做好科技金融大文章推动银行业加大科技型企业金融供给与服务中国人民银行金融监管总局等部门于近日联合印发关于扎实做好科技金融大文章的工作方案围绕培育支持科技创新的金融市场生态提出一系列有针对性的工作举措此外今年5月份金融监管总局发布关于银行业保险业做好金融五篇大文章的指导意见提出未来5年银行业保险业多层次广覆盖多样化可持续的五篇大文章服务体系基本形成相关工作机制更加完善产品更加丰富今年1月份金融监管总局发布关于加强科技型企业全生命周期金融服务的通知引导银行机构把更多金融资源用于促进科技创新不断提升金融支持科技型企业质效中国银行研究院研究员叶银丹表示通过一系列金融监管政策的引导和激励如税收优惠融资对接等措施为科创企业融资提供了良好的政策环境有助于吸引更多的金融资源投入科技创新领域增强金融支持精准性和可持续性加速推进科技成果转化和科创产业进程金融监管部门和银行机构深化金融供给侧结构性改革切实把五篇大文章落地落细提高金融服务实体经济的质量和水平2024年一季度金融机构贷款投向统计报告显示贷款支持科创企业力度较大2024年一季度末获得贷款支持的科技型中小企业2173万家获贷率479比上年末高11个百分点科技型中小企业本外币贷款余额27万亿元同比增长204比同期各项贷款增速高112个百分点兴业研究公司金融监管高级研究员陈昊表示对于金融机构自身而言在我国经济增长换挡的背景下银行业亟需寻找未来资产投放的蓄水池作为国家政策支持重点

### 批量计算相似度

#### 遍历+打印文件夹数据

In [12]:
def get_news_data(folder_path):
    news_data = {}
    
    # 遍历文件夹中的每个子文件夹（新闻类型文件夹）
    for news_type_folder in os.listdir(folder_path):
        news_type_path = os.path.join(folder_path, news_type_folder)
        if os.path.isdir(news_type_path):
            # 初始化新闻类型对应的新闻列表
            news_data[news_type_folder] = {}
            
            # 遍历每个新闻类型文件夹中的Word文件
            for doc_file in os.listdir(news_type_path):
                doc_file_path = os.path.join(news_type_path, doc_file)
                if doc_file_path.endswith('.docx'):
                    try:
                        content = read_word_doc(doc_file_path)
                        # 将新闻内容添加到新闻字典中
                        news_data[news_type_folder][doc_file] = content
                    except Exception as e:
                        print(f"Error reading {doc_file_path}: {e}")
    
    return news_data


In [13]:
folder_path = 'news'  # 替换为你的文件夹路径
news_data = get_news_data(folder_path)

# 美观地打印嵌套字典结果
import pprint
pprint.pprint(news_data)

{'体育': {},
 '国际': {},
 '时政': {'大规模宕机为全球信息技术安全敲响警钟.docx': '大规模宕机为全球信息技术安全敲响警钟新华社伦敦7月20日电记者郭爽19日微软视窗系统以及该公司其他部分应用和服务发生大规模宕机造成多国航空铁路海运金融医疗酒店等行业无法正常运转众多企业和个人用户的工作和生活受到严重干扰微软首席执行官萨蒂亚纳德拉当天在社交媒体X上发文确认为微软提供服务的安全技术企业众击公司发布的一项软件更新是造成这次全球性宕机的主要原因这起宕机事件的影响范围和严重程度十分罕见为各国政府行业和个人用户敲响警钟英国工程技术学会的网络安全专家朱奈德阿里指出这次宕机的规模可能史无前例对全球信息技术IT行业团队构成了重大挑战但同时也为软件工程专业人员提供重要经验完全消除影响仍需时间据外媒报道总部位于美国的众击公司在全球拥有超过2万客户其中包括微软和亚马逊等科技巨头该公司首席执行官乔治库尔茨19日在社交媒体X上发文说此次事件不涉及网络攻击而是源自该公司为微软视窗系统发布的软件更新中存在缺陷该问题已被识别隔离并已部署修复措施库尔茨当天还在媒体采访中说就我们给客户旅行者以及所有受波及者造成的影响深表歉意公司正在努力解决问题但一些系统可能需要一些时间才能从此次故障中恢复过来虽然众击公司已经与微软合作迅速恢复大部分服务但专家认为需要进一步评估这次宕机事件的长期影响英国计算机学会网络安全专家亚当史密斯指出修复程序必须应用于世界各地的大量计算机这需要一段时间但如果计算机进入蓝屏和无限循环恢复可能会更困难需要几天甚至几周时间朱奈德阿里认为众击公司正将此事件列为最优先事项加以解决这次宕机的长期影响尚未被完全理解但它们将影响到未来关键安全更新的及时采用对IT系统风险保持警惕专家认为宕机事件凸显全球互联网基础设施的脆弱性需要对IT系统的复杂性以及各领域高度依赖网络基础设施的潜在风险保持警惕英国工程技术学会专家伊恩科登说世界各地发生的重大IT系统中断事件反映了经济国防和国家安全等方面对数字服务依赖日益增加的问题也因此凸显数字服务安全和韧性的重要性英国布里斯托尔大学计算机科学学院专家奥莫罗尼亚认为需要时刻警惕我们每天依赖的云基础设施和其他关键系统如今的网络基础设施非常复杂对其依赖性很广泛而对负责构建这些基础设施的人来说这些风险往往并不明显本次事件也存在大众尚不明了的复杂情

In [14]:
# 遍历并打印每种新闻类型文件夹中的Word文档名
for news_type, docs in news_data.items():
    print(f"News type: {news_type}")
    for doc_file in docs.keys():
        print(f"  - Document name: {doc_file}")

News type: 体育
News type: 国际
News type: 时政
  - Document name: 大规模宕机为全球信息技术安全敲响警钟.docx


#### 文心一言API调用

In [ ]:
API_Key = 'ARTiXOfYDoIwrBhRo0FLRSoT'
Secret_Key = 'BvHofSd0ENnmKkRID1IYR0lgbO1VP3aT'


In [38]:
import requests
import json


def main():
        
    url = "https://aip.baidubce.com/oauth/2.0/token?client_id=ARTiXOfYDoIwrBhRo0FLRSoT&client_secret=BvHofSd0ENnmKkRID1IYR0lgbO1VP3aT&grant_type=client_credentials"
    
    payload = json.dumps("")
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    
    response = requests.request("POST", url, headers=headers, data=payload)
    
    print(response.text)
    

if __name__ == '__main__':
    main()


{"refresh_token":"25.c3c82ef1b303b11b6969bcd55922837c.315360000.2037018086.282335-81691303","expires_in":2592000,"session_key":"9mzdA86edtC+hRM0cyd8X7ZQqNiLehwh7O4+ZGU8X8CTNRJUgQaX5ho\/\/LqSR7M\/8FfitZ1UGCeFxZgEldad6O1zp1LGMw==","access_token":"24.8d169516360fc523f7a6659dcc2486d7.2592000.1724250086.282335-81691303","scope":"public brain_all_scope wenxinworkshop_mgr ai_custom_yiyan_com ai_custom_yiyan_com_eb_instant ai_custom_yiyan_com_bloomz7b1 ai_custom_yiyan_com_emb_text ai_custom_yiyan_com_llama2_7b ai_custom_yiyan_com_llama2_13b ai_custom_yiyan_com_llama2_70b ai_custom_yiyan_com_chatglm2_6b_32k ai_custom_yiyan_com_aquilachat_7b ai_custom_yiyan_com_emb_bge_large_zh ai_custom_yiyan_com_emb_bge_large_en ai_custom_yiyan_com_qianfan_chinese_llama_2_7b ai_custom_qianfan_bloomz_7b_compressed ai_custom_yiyan_com_eb_pro ai_custom_yiyan_com_adv_pro ai_custom_yiyan_com_sd_xl ai_custom_yiyan_com_tokenizer_eb ai_custom_yiyan_com_ai_apaas ai_custom_yiyan_com_qf_chinese_llama_2_13b ai_custom_yiya